In [7]:
import pandas as pd
import numpy as np
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [8]:
df = pd.read_csv('data/text-preprocessing.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df[df['author_location'].isnull() == False]
df = df[df['author_location'].isna() == False]

df = df.head(10)
df.head(5)

,tweet_id,author_name,author_username,author_location,content,content_clean,like_count,retweet_count,reply_count,source,keyword,address,longitude,created_at,latitude,content_token,tweet_tokens_fdist,tweet_tokens_WSW,tweet_tokens_stemmed
0,153619597713608704,MariBerlomba,MariBerlomba,di dalam asa pemuda,"1. lomba poster ilmiah ""energi baru/terbarukan...",lomba poster ilmiah energi baruterbarukan dead...,0.0,1.0,0.0,Twitter for BlackBerry®,energi baru terbarukan,0,0.000000,2012-01-01 23:32:20,0.000000,"['lomba', 'poster', 'ilmiah', 'energi', 'barut...",<FreqDist with 11 samples and 11 outcomes>,"['lomba', 'poster', 'ilmiah', 'energi', 'barut...","['lomba', 'poster', 'ilmiah', 'energi', 'barut..."
1,153857491925610496,vegaaulia,vegaaulia,"Bogor, Indonesia",elaahhh ini si bapak tiba2 ngomong soal energi...,elaahhh ini si bapak tiba ngomong soal energi ...,0.0,0.0,1.0,Mobile Web,energi baru terbarukan,"Bogor, West Java, 16121, Indonesia",106.797242,2012-01-02 15:17:39,-6.596299,"['elaahhh', 'ini', 'si', 'bapak', 'tiba', 'ngo...",<FreqDist with 13 samples and 13 outcomes>,"['elaahhh', 'ngomong', 'energi', 'terbarukan',...","['elaahhh', 'ngomong', 'energi', 'baru', 'tv',..."
2,154361424154603520,enbater,enbater,Indonesia,masih banyak daerah terpencil membutuhkam list...,masih banyak daerah terpencil membutuhkam list...,0.0,0.0,0.0,MOTOBLUR,energi baru terbarukan,Indonesia,117.890285,2012-01-04 00:40:06,-2.483383,"['masih', 'banyak', 'daerah', 'terpencil', 'me...",<FreqDist with 13 samples and 13 outcomes>,"['daerah', 'terpencil', 'membutuhkam', 'listri...","['daerah', 'pencil', 'membutuhkam', 'listrik',..."
3,156397219346518017,imronrosidin,imronrosidin,"ÜT: -6.895087,107.627799",gw kira kalo @provocactive gak tertarik dgn ma...,gw kira kalo gak tertarik dgn materi energi ba...,0.0,0.0,1.0,UberSocial for BlackBerry,energi baru terbarukan,0,0.000000,2012-01-09 15:29:37,0.000000,"['gw', 'kira', 'kalo', 'gak', 'tertarik', 'dgn...",<FreqDist with 17 samples and 17 outcomes>,"['tertarik', 'materi', 'energi', 'baruterbaruk...","['tarik', 'materi', 'energi', 'baruterbarukan'..."
4,156936339718279168,revozone,revozone,Indonesia,satu lagi yang aneh. inget jatropa? alias miny...,satu lagi yang aneh inget jatropa alias minyak...,0.0,0.0,0.0,UberSocial for BlackBerry,energi baru terbarukan,Indonesia,117.890285,2012-01-11 03:11:53,-2.483383,"['satu', 'lagi', 'yang', 'aneh', 'inget', 'jat...",<FreqDist with 19 samples and 20 outcomes>,"['aneh', 'inget', 'jatropa', 'alias', 'minyak'...","['aneh', 'inget', 'jatropa', 'alias', 'minyak'..."


In [9]:
def lookup_location(location):
    geo_locator = Nominatim(user_agent="LearnPython")
    try:
        location = geo_locator.geocode(location, language='en')
    except GeocoderTimedOut:
        return None
    except:
        return None
    return location

In [10]:
def lookup_location_by_coordinate(latitude, longitude):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        location = geolocator.reverse([latitude+","+longitude])
    except GeocoderTimedOut:
        return None
    except:
        return None
    return location

In [17]:
for index, row in df.iterrows():
    if row['author_location']:
        location = lookup_location(row['author_location'])
        if location is not None:
            df.loc[index, 'address'] = location.address
            df.loc[index, 'latitude'] = location.latitude
            df.loc[index, 'longitude'] = location.longitude

            # coordinate
            location_detail = lookup_location_by_coordinate(location.latitude,location.longitude )
            if location_detail:
                address = location_detail.raw['address']
                country = address.get('country', '')
                province = address.get('state', '')
                city = address.get('city', '')
                
                if country:
                    print(country, province, city)
                    df.loc[index, 'country'] = country
                    df.loc[index, 'province'] = province
                    df.loc[index, 'city'] = city
        else:
            df.loc[index, 'address'] = ""

authroo location di dalam asa pemuda
None
authroo location Bogor, Indonesia
None
authroo location Indonesia
None
authroo location ÜT: -6.895087,107.627799
None
authroo location Indonesia
None
authroo location Indonesia
None
authroo location 0
None
authroo location Bekasi, Jawa Barat
None
authroo location Indonesia - Jakarta
None
authroo location 0
None


In [ ]:
df.to_csv('data-energi-baru-terbarukan.csv')